# **Importany `package` to install**

In [1]:
# !pip install transformers datasets huggingface_hub
# !pip install transformers
# !pip install langchain-community
# !pip install datasets


# # !pip install -q youtube-transcript-api langchain-community langchain-openai \
# #                faiss-cpu tiktoken python-dotenv


!pip install datasets langchain langchain-community faiss-cpu
!pip install faiss-cpu
!pip install -U datasets fsspec
# !pip install huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fssp

In [2]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
#

In [3]:
# from huggingface_hub import notebook_login
# notebook_login()
# !huggingface-cli login

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# **Load Dataset**

In [5]:
# pip install datasets
from datasets import load_dataset


ds = load_dataset("Abirate/english_quotes")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [6]:

print(ds['train'][3])


{'quote': '“So many books, so little time.”', 'author': 'Frank Zappa', 'tags': ['books', 'humor']}


# **Text splitter**

In [47]:
all_transcripts = [
    " ".join(str(value) for value in item.values())
    for item in ds["train"]]

# for i in range(3):
#     print(all_transcripts[i])
# all_transcripts

In [48]:

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents(all_transcripts)

In [50]:
from langchain.embeddings import HuggingFaceEmbeddings

# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# embeddings = HuggingFaceEmbeddings(model="text-embedding-3-small")
# vector_store = FAISS.from_documents(chunks, embeddings)




model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
vector_store = FAISS.from_documents(chunks, embeddings)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})



# **Retriverl**

In [17]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [53]:
# retriever.invoke('“Quotes about insanity attributed to Einstein”')
first_doc = retriever.invoke(" Einstein")
print(first_doc)

[Document(id='3d0007d6-b4a7-4bdc-a026-bfe4621324d3', metadata={}, page_content="“A clever person solves a problem. A wise person avoids it.” Albert Einstein ['misattributed-to-einstein']"), Document(id='e3570e14-419e-422f-a348-fe18e92adcec', metadata={}, page_content="“If I were not a physicist, I would probably be a musician. I often think in music. I live my daydreams in music. I see my life in terms of music.” Albert Einstein ['music']"), Document(id='541d2b60-d149-4218-9bb0-38b137706145', metadata={}, page_content="“I have no special talents. I am only passionately curious.” Albert Einstein ['inspirational-life']"), Document(id='cc7b1881-bdfd-495d-a510-021b3861683d', metadata={}, page_content="“Everything must be made as simple as possible. But not simpler.” Albert Einstein ['einstein', 'paraphrased', 'science', 'systems']")]


In [24]:
from transformers import pipeline


from langchain.llms import HuggingFacePipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"



llm_pipeline = pipeline("text-generation", model=model_name)
llm = HuggingFacePipeline(pipeline=llm_pipeline)


Device set to use cuda:0


# **Creating Prompt**

In [90]:
prompt = PromptTemplate (
    template="""
      {context}

      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [91]:
question          = '“Quotes about insanity attributed to Einstein”'
retrieved_docs = retriever.invoke(question)




In [92]:
retrieved_docs

[Document(id='11ed6b50-2017-48d1-996f-8605de06a9af', metadata={}, page_content="“A question that sometimes drives me hazy: am I or are the others crazy?” Albert Einstein ['crazy', 'question', 'thought-provoking']"),
 Document(id='7f4ee956-e3dc-41dc-aa9e-1c87ea556a6b', metadata={}, page_content="“The world is a dangerous place to live, not because of the people who are evil, but because of the people who don't do anything about it.” Albert Einstein ['dangerous', 'evil', 'people', 'world']"),
 Document(id='a5d7e121-2698-4593-b630-b3540654f9e0', metadata={}, page_content="“In individuals, insanity is rare; but in groups, parties, nations and epochs, it is the rule.” Friedrich Nietzsche ['insanity', 'society']"),
 Document(id='33200b48-81ec-4035-925d-94d07c81c6bb', metadata={}, page_content="“I became insane, with long intervals of horrible sanity.” Edgar Allan Poe ['insanity', 'sanity']")]

In [93]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"“A question that sometimes drives me hazy: am I or are the others crazy?” Albert Einstein ['crazy', 'question', 'thought-provoking']\n\n“The world is a dangerous place to live, not because of the people who are evil, but because of the people who don't do anything about it.” Albert Einstein ['dangerous', 'evil', 'people', 'world']\n\n“In individuals, insanity is rare; but in groups, parties, nations and epochs, it is the rule.” Friedrich Nietzsche ['insanity', 'society']\n\n“I became insane, with long intervals of horrible sanity.” Edgar Allan Poe ['insanity', 'sanity']"

In [94]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [95]:
final_prompt

StringPromptValue(text="\n      “A question that sometimes drives me hazy: am I or are the others crazy?” Albert Einstein ['crazy', 'question', 'thought-provoking']\n\n“The world is a dangerous place to live, not because of the people who are evil, but because of the people who don't do anything about it.” Albert Einstein ['dangerous', 'evil', 'people', 'world']\n\n“In individuals, insanity is rare; but in groups, parties, nations and epochs, it is the rule.” Friedrich Nietzsche ['insanity', 'society']\n\n“I became insane, with long intervals of horrible sanity.” Edgar Allan Poe ['insanity', 'sanity']\n\n      Question: “Quotes about insanity attributed to Einstein”\n    ")

In [96]:
# answer = llm.invoke(final_prompt)
# print(answer.content)
# print(answer.conten////t)


answer = llm.invoke(final_prompt)


In [97]:
type(answer)

str

```

# *Building a Chain*

In [98]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

def format_docs(retrieved_docs):
  context_text = "\ n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text




parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})




parallel_chain.invoke('what is Love')

{'context': "“Love is not affectionate feeling, but a steady wish for the loved person's ultimate good as far as it can be obtained.” C.S. Lewis ['love']\\ n\n“Love is an untamed force. When we try to control it, it destroys us. When we try to imprison it, it enslaves us. When we try to understand it, it leaves us feeling lost and confused.” Paulo Coelho ['love']\\ n\n“Love is always patient and kind. It is never jealous. Love is never boastful or conceited. It is never rude or selfish. It does not take offense and is not resentful. Love takes no pleasure in other peopleâ€™s sins, but delights in the truth. It is always ready to excuse, to trust, to hope, and to endure whatever comes.” Anonymous, ['1-corinthians-13', 'love']\\ n\n“Love is patient, love is kind. It does not envy, it does not boast, it is not proud. It is not rude, it is not self-seeking, it is not easily angered, it keeps no record of wrongs. Love does not delight in evil but rejoices with the truth. It always protects,

In [99]:
parser = StrOutputParser()

In [100]:
main_chain = parallel_chain |prompt | llm | parser

In [101]:
main_chain.invoke('Give me best of quore on love')



"\n      “The best love is the kind that awakens the soul and makes us reach for more, that plants a fire in our hearts and brings peace to our minds. And that's what you've given me. That's what I'd hoped to give you forever” Nicholas Sparks ['love', 'nicholas-sparks', 'romance', 'the-notebook']\\ n\n“Love is not affectionate feeling, but a steady wish for the loved person's ultimate good as far as it can be obtained.” C.S. Lewis ['love']\\ n\n“Love is an untamed force. When we try to control it, it destroys us. When we try to imprison it, it enslaves us. When we try to understand it, it leaves us feeling lost and confused.” Paulo Coelho ['love']\\ n\n“You don't love someone because they're perfect, you love them in spite of the fact that they're not.” Jodi Picoult, ['imperfect', 'love']\n\n      Question: Give me best of quore on love\n     A poem is a flower, an ode is a song\n     A poem is a tale, a song is a story\n     A poem is a tale, a song is a lullaby\n     A poem is a tale

In [102]:
pip install gradio


In [103]:
import gradio as gr

# Ensure main_chain exists; replace this with your actual chain logic
try:
    main_chain
except NameError:
    from langchain.chains import LLMChain  # adjust import if needed
    main_chain = None  # Replace this with the actual chain like: main_chain = LLMChain(...)

# Function to generate an answer
def generate_answer(question):
    if question.strip() == "":
        return "Please enter a question."
    if main_chain is None:
        return "main_chain is not defined. Please configure your LangChain pipeline."
    return main_chain.invoke(question)

# Launch Gradio interface
gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(label="Enter your question:", placeholder="Type your question here...", lines=1),
    outputs=gr.Textbox(label="Answer"),
    title="Question Answering App",
    description="Enter a question and get a simple answer."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b0040c1f48cbfb7c62.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
